In [1]:
import folium
import pandas as pd
from folium.plugins import HeatMap
from google.colab import files

In [2]:
upload = files.upload()

Saving AEGISDataset.csv to AEGISDataset.csv


In [3]:
data = pd.read_csv("AEGISDataset.csv")

In [4]:
data.head()

lat         lon  flood_heig  elevation  precipitat
0  14.640394  121.055708           0  54.553295         9.0
1  14.698299  121.002132           0  21.856272        10.0
2  14.698858  121.100261           0  69.322807        16.0
3  14.571310  120.983334           0  10.987241         8.0
4  14.762232  121.075735           0  87.889847        18.0

In [5]:
numeric_columns = ["lat", "lon", "flood_heig", "elevation", "precipitat"]
data[numeric_columns] = data[numeric_columns].apply(pd.to_numeric, errors='coerce')
data_clean = data.dropna()
print(data_clean.isnull().sum())

lat           0
lon           0
flood_heig    0
elevation     0
precipitat    0
dtype: int64


In [6]:
data_clean["normalized_flood_height"] = (data_clean['flood_heig'] - data['flood_heig'].min()) / (data_clean['flood_heig'].max() - data_clean["flood_heig"].min())
data_clean["normalized_elevation"] = (data_clean['elevation'] - data['elevation'].min()) / (data_clean['elevation'].max() - data_clean["elevation"].min())
data_clean["normalized_precipitat"] = (data_clean['precipitat'] - data['precipitat'].min()) / (data_clean['precipitat'].max() - data_clean["precipitat"].min())

In [7]:
data_clean['normalized_elevation']  = 1 - data_clean['normalized_elevation']

In [8]:
data_clean['risk_score'] = (data_clean['normalized_flood_height'] * 0.4 + data_clean['normalized_elevation'] * 0.3 + data_clean['normalized_precipitat'] * 0.3)

In [9]:
print("\nRisk score for Ech Location:")
for index, row in data_clean.iterrows():
  print(f"Latitude: {row['lat']}, Longitude: {row['lon']}, Flood Height: {row['flood_heig']}, Risk Score: {row['risk_score']}")


Risk score for Ech Location:
Latitude: 14.64039374, Longitude: 121.0557085, Flood Height: 0.0, Risk Score: 0.2503582609426578
Latitude: 14.69829935, Longitude: 121.0021322, Flood Height: 0.0, Risk Score: 0.3714576900771931
Latitude: 14.69885841, Longitude: 121.1002614, Flood Height: 0.0, Risk Score: 0.30210963466319884
Latitude: 14.57131006, Longitude: 120.9833345, Flood Height: 0.0, Risk Score: 0.3783929056101954
Latitude: 14.7622323, Longitude: 121.0757349, Flood Height: 0.0, Risk Score: 0.2700267817151137
Latitude: 14.6011784, Longitude: 121.0885987, Flood Height: 0.0, Risk Score: 0.4098931394636641
Latitude: 14.70803089, Longitude: 121.052674, Flood Height: 0.0, Risk Score: 0.3244220322521432
Latitude: 14.70920612, Longitude: 121.0502922, Flood Height: 0.0, Risk Score: 0.3445020220605377
Latitude: 14.70962531, Longitude: 121.0470059, Flood Height: 0.0, Risk Score: 0.3303591941495462
Latitude: 14.66933825, Longitude: 121.1328981, Flood Height: 0.0, Risk Score: 0.38514977754177726
L

In [12]:
def get_color(risk_score):
  if risk_score < 0.2:
    return 'green'
  elif 0.2 <= risk_score < 0.4:
    return 'yellow'
  elif 0.4 <= risk_score < 0.6:
    return 'orange'
  elif 0.6 <= risk_score < 0.8:
    return 'red'
  else:
    return 'darkred'

data_clean['risk_color'] = data_clean['risk_score'].apply(get_color)

ontario_coordinates = [52.008463631022224, -85.18766469281854]
map_flood_risk = folium.Map(location=ontario_coordinates, zoom_start=12)
heat_data = data_clean[['lat', 'lon', 'risk_score']].values.tolist()
HeatMap(heat_data, radius=15, max_zoom=13).add_to(map_flood_risk)
print('\nRisk Score for Each Location')
for index, row in data_clean.iterrows():
  print(f"Latitude: {row['lat']}, Longitude: {row['lon']}, Flood Height: {row['flood_heig']}, Risk Score: {row['risk_score']}")
map_flood_risk.save('ontario_flood_risk_heatmap.html')
map_flood_risk


Risk Score for Each Location
Latitude: 14.64039374, Longitude: 121.0557085, Flood Height: 0, Risk Score: 0.2503582609426578
Latitude: 14.69829935, Longitude: 121.0021322, Flood Height: 0, Risk Score: 0.3714576900771931
Latitude: 14.69885841, Longitude: 121.1002614, Flood Height: 0, Risk Score: 0.30210963466319884
Latitude: 14.57131006, Longitude: 120.9833345, Flood Height: 0, Risk Score: 0.3783929056101954
Latitude: 14.7622323, Longitude: 121.0757349, Flood Height: 0, Risk Score: 0.2700267817151137
Latitude: 14.6011784, Longitude: 121.0885987, Flood Height: 0, Risk Score: 0.4098931394636641
Latitude: 14.70803089, Longitude: 121.052674, Flood Height: 0, Risk Score: 0.3244220322521432
Latitude: 14.70920612, Longitude: 121.0502922, Flood Height: 0, Risk Score: 0.3445020220605377
Latitude: 14.70962531, Longitude: 121.0470059, Flood Height: 0, Risk Score: 0.3303591941495462
Latitude: 14.66933825, Longitude: 121.1328981, Flood Height: 0, Risk Score: 0.38514977754177726
Latitude: 14.72701256